In [1]:
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler
from logging.handlers import RotatingFileHandler
from sklearn.externals import joblib
from collections import OrderedDict
import sys
from dayu.hooks.oss_hook import OSSHook

from dayu.hooks.hive_server_hook import HiveServerHook
from dayu.hooks.hive_cli_hook import HiveCliHook

import pandas as pd 
import numpy as np
import time
import requests

def split_table_name(datain):
    new_cols = []
    for column in datain.columns:
        if(len(column.split('.'))<2):
            return datain
        tb_name, col_name = column.split('.')
        new_cols.append((column, col_name))
    datain = datain.rename(columns=dict(new_cols))
    return datain

def read_from_hive2(output_file_name,insql,dtype):
    filename = output_file_name
    filepath = curr_dir+filename
    hive = HiveServerHook("warehouse_hive")
    hive.to_csv(insql,filepath , delimiter=',',lineterminator='\n', output_header=True)
    outdata = pd.read_csv(filepath, header=0,dtype=dtype)
    # 去除列名中带有的表名
#     outdata = split_table_name(outdata)
    return outdata

def date_time_sub(startTime,endTime,date_format):
    try:
        startTime= datetime.datetime.strptime(startTime,date_format)
        endTime= datetime.datetime.strptime(endTime,date_format)
        return (endTime - startTime).days
    except:
        pass
curr_date = str(datetime.datetime.now())[0:10]
logging.info('当前日期: %s'%(curr_date))
curr_dir = '/home/souche/qiongjiu/valuation/'
curr_dir = '/home/souche/qiongjiu/hgc/'
curr_date=str(datetime.datetime.now())[0:10]
hive_cli = HiveCliHook("warehouse_hive")

DAYU_HOME : /home/souche/projects/datacenter-etl-v2


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[2021-01-05 15:42:51,299] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
[2021-01-05 15:42:51,328] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
[2021-01-05 15:42:51,774] {<ipython-input-1-1cc0538bb418>:46} INFO - 当前日期: 2021-01-05


In [2]:
sql_info = """
SELECT
    t1.order_id,
    t1.c_goods_id,
    t1.pay_time,
    t2.c_trade_code 
FROM
(
    SELECT
        order_id,
        c_goods_id,
        total_price order_price,
        seller_price,
        order_status,
        auction_mode,
        goods_source,
        create_time pay_time
    FROM
        edw_cyp.dwb_evt_tra_cypf_order_dd
    WHERE
        ds = '2021-01-04' AND
        c_goods_id IS NOT NULL AND
        create_time >= '2016-06-01' and order_status = '80' 
) t1
JOIN
(
    SELECT
        * 
    FROM
        edw_cyp.dwb_gds_cypd_goods_dd
    WHERE
        ds ='2021-01-04'
       and c_id is not null 
) t2
ON t1.c_goods_id = t2.c_id 
left join (select model_code,
                  guide_price 
           from db_data.ods_car_model_model_price 
           where ds ='2021-01-04') t3 
on t2.c_car_model_id = t3.model_code 
WHERE
    t2.c_register_date IS NOT NULL AND
    t2.c_mileage IS NOT NULL 
    and t1.order_status = '80' 
"""
dtype={'city_code':str}
order_trade_code = read_from_hive2('order_trade_code',sql_info,dtype)



[2021-01-05 15:42:51,837] {hiveserver2:138} INFO - Using database default as default
[2021-01-05 15:42:51,987] {hive_server_hook:112} INFO - Running query: 
SELECT
    t1.order_id,
    t1.c_goods_id,
    t1.pay_time,
    t2.c_trade_code 
FROM
(
    SELECT
        order_id,
        c_goods_id,
        total_price order_price,
        seller_price,
        order_status,
        auction_mode,
        goods_source,
        create_time pay_time
    FROM
        edw_cyp.dwb_evt_tra_cypf_order_dd
    WHERE
        ds = '2021-01-04' AND
        c_goods_id IS NOT NULL AND
        create_time >= '2016-06-01' and order_status = '80' 
) t1
JOIN
(
    SELECT
        * 
    FROM
        edw_cyp.dwb_gds_cypd_goods_dd
    WHERE
        ds ='2021-01-04'
       and c_id is not null 
) t2
ON t1.c_goods_id = t2.c_id 
left join (select model_code,
                  guide_price 
           from db_data.ods_car_model_model_price 
           where ds ='2021-01-04') t3 
on t2.c_car_model_id = t3.model_code 
WH

/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
## 处理列名称
order_trade_code.columns = [col.split('.')[1] for col in order_trade_code.columns]


In [4]:
order_trade_code = order_trade_code.loc[(order_trade_code['pay_time'] >= '2020-12-01') & 
                                       (order_trade_code['pay_time'] <= '2020-12-31')]


In [5]:
order_trade_code.shape

(7025, 4)

In [6]:
def one_thread_fun(threadName,data_df):
    data_df_list = []
    i = 0
    for data in data_df.to_dict(orient='records'):
        i = i+1
        if i % 50 == 0:
            print(threadName+'....已完成数据....',i)
        try:
            params={}
            params['reportVersion']= 'shdv003'
            params['regenerate']= '0'
            params['reportCode']= data['c_trade_code']

            req = requests.get('https://npi.cheyipai.com/detect/report/reportApi/getMerchatReportViewModel?',params=params)
            data_r = req.json()
#             print(data_r)
            re_data = {}
            re_data['trade_code'] = data['c_trade_code']
            if 'basicInfos' in data_r['data'].keys() and data_r['data']['basicInfos'] != None :
                re_data['车况提示'] = data_r['data']['basicInfos'][0]['basicItemInfos'][0]['fieldValue']
                re_data['买方负责'] = data_r['data']['basicInfos'][2]['basicItemInfos'][0]['fieldValue']
                re_data['卖方责任'] = data_r['data']['basicInfos'][2]['basicItemInfos'][1]['fieldValue']
                if len(data_r['data']['basicInfos']) > 3:
                    for d_one in data_r['data']['basicInfos'][3]['basicItemInfos']:
                        re_data[d_one['fieldCode']] = d_one['fieldValue']
                if len(data_r['data']['basicInfos']) > 4:
                    for d_one in data_r['data']['basicInfos'][4]['basicItemInfos']:
                        re_data[d_one['fieldCode']] = d_one['fieldValue']
                if len(data_r['data']['basicInfos']) > 5: 
                    for d_one in data_r['data']['basicInfos'][5]['basicItemInfos']:
                        re_data[d_one['fieldName']] = d_one['fieldValue']
                if len(data_r['data']['basicInfos']) > 1:      
                    for d_3 in data_r['data']['basicInfos'][1]['basicItemInfos']:
                        re_data[d_3['fieldName']] = d_3['fieldValue']

            itemDesc = []
            for data in data_r['data']['photoInfos']['photoList']:
                if 'itemList' in data.keys() and data['itemList'] != None:
                    for d_0 in data['itemList']:
                        itemDesc.append(d_0['itemName']+ " "+ d_0['itemDesc'])
                if 'photoList' in data.keys() and data['photoList'] != None:
                    for d_1 in  data['photoList']:
                        if d_1['photoDesc'] != None:
                            if len(d_1['photoDesc'].split(" ")) >1:
                                itemDesc.append(d_1['photoDesc'])
            re_data['检测明细描述'] = ';'.join(itemDesc)

            if  data_r['data']['maintenanceInfo'] != None:
                re_data['mileageCount'] = data_r['data']['maintenanceInfo']['mileageCount']

            if data_r['data']['evaluateInfo'] != None:
                re_data['rank'] = data_r['data']['evaluateInfo']['rank']
                re_data['score'] = data_r['data']['evaluateInfo']['score']
                re_data['scoreDesc'] = data_r['data']['evaluateInfo']['scoreDesc']
                re_data['rankDesc'] = data_r['data']['evaluateInfo']['rankDesc']
            data_df_list.append(re_data)
        except:
            data_df_list.append(re_data)
            pass
    
    data_df_df= pd.DataFrame(data_df_list)
    data_df_df.to_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')

In [7]:
order_trade_code['c_trade_code_3'] = order_trade_code['c_trade_code'].map(lambda x:x[:3])
order_trade_code = order_trade_code.loc[~(order_trade_code['c_trade_code_3'] == 'MTR')]


In [8]:
order_trade_code['c_trade_code_3'].unique()

array(['DY2', 'DM2', 'DQ2'], dtype=object)

In [9]:
order_trade_code['pay_time'] = order_trade_code['pay_time'].map(lambda x:x[:10])
# order_trade_code_20200816_20200926 = order_trade_code
# # order_trade_code_2020.shape
# # order_trade_code_2020 = order_trade_code.loc[(order_trade_code['pay_time'] >='2020-01-01')]
# order_trade_code_20200816_20200926.shape
order_trade_code.shape


(7025, 5)

In [11]:
order_trade_code['c_trade_code'].map(lambda x:x[:3]).unique()

array(['DY2', 'DM2', 'DQ2'], dtype=object)

In [12]:
# 创建两个线程
# import _thread
# try:
#     sep_num = int(order_trade_code_2020.shape[0] / 50)
#     for i in range(50):
#         threadName = 'threadName_1_2019_' + str(i)
#         one_data = order_trade_code_2020[sep_num*i:sep_num*(i+1)] 
#         _thread.start_new_thread(one_thread_fun, ("Thread_1_2019_"+str(i), one_data))
# except:
#     print ("Error: 无法启动线程")

# while 1:
#     pass

try:
    sep_num = int(order_trade_code.shape[0] / 5)
    for i in range(0,5):
        one_data = order_trade_code[sep_num*i:sep_num*(i+1)] 
        one_thread_fun("Thread_0_20201201_20201231_"+str(i), one_data)
except:
    print ("Error: 无法启动线程")



Thread_0_20201201_20201231_0....已完成数据.... 50
Thread_0_20201201_20201231_0....已完成数据.... 100
Thread_0_20201201_20201231_0....已完成数据.... 150
Thread_0_20201201_20201231_0....已完成数据.... 200
Thread_0_20201201_20201231_0....已完成数据.... 250
Thread_0_20201201_20201231_0....已完成数据.... 300
Thread_0_20201201_20201231_0....已完成数据.... 350
Thread_0_20201201_20201231_0....已完成数据.... 400
Thread_0_20201201_20201231_0....已完成数据.... 450
Thread_0_20201201_20201231_0....已完成数据.... 500
Thread_0_20201201_20201231_0....已完成数据.... 550
Thread_0_20201201_20201231_0....已完成数据.... 600
Thread_0_20201201_20201231_0....已完成数据.... 650
Thread_0_20201201_20201231_0....已完成数据.... 700
Thread_0_20201201_20201231_0....已完成数据.... 750
Thread_0_20201201_20201231_0....已完成数据.... 800
Thread_0_20201201_20201231_0....已完成数据.... 850
Thread_0_20201201_20201231_0....已完成数据.... 900
Thread_0_20201201_20201231_0....已完成数据.... 950
Thread_0_20201201_20201231_0....已完成数据.... 1000
Thread_0_20201201_20201231_0....已完成数据.... 1050
Thread_0_20201201_20201231_0....已

In [20]:
i = 0
sep_num = int(order_trade_code_2020.shape[0] / 100)
order_trade_code_2020[sep_num*i:sep_num*(i+1)]


,order_id,c_goods_id,pay_time,c_trade_code
5460,01010200107200348,9010191029900159,2020-01-07,Y191029115412001
5610,01010200111200084,9010191121300329,2020-01-11,Y191121144928001
5734,01010200507200010,9010191211300380,2020-05-07,Y191211145919001
5790,01010200619600077,9010191219900185,2020-06-19,Y191219114528001
5865,01010200102200300,9010191228900404,2020-01-02,M191228143404001
5874,01010200102500032,9010191230300059,2020-01-02,N191230095755001
5875,01010200102200028,9010191230300249,2020-01-02,N191230132740001
5881,01010191231500097,9010191231300089,2020-01-02,M191230161520001
5885,01010200109500220,9010191231300514,2020-01-09,Y191231195814001
5888,01010200102200183,9010191231900463,2020-01-02,M191231161653001


In [15]:
threadName = 'Thread_2019_0'
data_df = pd.read_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')
print(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')

/home/souche/qiongjiu/hgc//data_df_df_gujia_Thread_2019_0.xlsx


In [17]:
check_data_null = pd.read_excel("check_data_null.xlsx")
check_data_null['c_trade_code'] = check_data_null['trade_code']

In [18]:
check_data_null.shape

(3089, 9)

In [19]:
sep_num = int(check_data_null.shape[0] / 5)
for i in range(0,5):
    one_data = check_data_null[sep_num*i:sep_num*(i+1)] 
    one_thread_fun("Thread_0_check_data_null_1_"+str(i), one_data)


Thread_0_check_data_null_1_0....已完成数据.... 50
Thread_0_check_data_null_1_0....已完成数据.... 100
Thread_0_check_data_null_1_0....已完成数据.... 150
Thread_0_check_data_null_1_0....已完成数据.... 200
Thread_0_check_data_null_1_0....已完成数据.... 250
Thread_0_check_data_null_1_0....已完成数据.... 300
Thread_0_check_data_null_1_0....已完成数据.... 350
Thread_0_check_data_null_1_0....已完成数据.... 400
Thread_0_check_data_null_1_0....已完成数据.... 450
Thread_0_check_data_null_1_0....已完成数据.... 500
Thread_0_check_data_null_1_0....已完成数据.... 550
Thread_0_check_data_null_1_0....已完成数据.... 600
Thread_0_check_data_null_1_1....已完成数据.... 50
Thread_0_check_data_null_1_1....已完成数据.... 100
Thread_0_check_data_null_1_1....已完成数据.... 150
Thread_0_check_data_null_1_1....已完成数据.... 200
Thread_0_check_data_null_1_1....已完成数据.... 250
Thread_0_check_data_null_1_1....已完成数据.... 300
Thread_0_check_data_null_1_1....已完成数据.... 350
Thread_0_check_data_null_1_1....已完成数据.... 400
Thread_0_check_data_null_1_1....已完成数据.... 450
Thread_0_check_data_null_1_1....已完成数

In [4]:
try:
    sep_num = int(check_data_null.shape[0] / 5)
    for i in range(0,5):
        one_data = check_data_null[sep_num*i:sep_num*(i+1)] 
        one_thread_fun("Thread_0_check_data_null_"+str(i), one_data)
except:
    print ("Error: 无法启动线程")


Error: 无法启动线程


In [12]:
data_df_list = []
for i in range(5):
    threadName = 'Thread_0_check_data_null_' + str(i)
    data_df = pd.read_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')
    data_df_list.append(data_df)

data_df_list_df = pd.concat(data_df_list,axis=0)

/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [16]:
data_df_list_df[['trade_code','检测明细描述','rank','score']]


,trade_code,检测明细描述,rank,score
0,CRP160221000220,右后翼子板 重做漆;右前翼子板 重做漆;左后翼子板 重做漆;行李箱盖 螺丝松动;右前车门封边...,NaN,NaN
1,CRP160321000977,钥匙 破损;前保险杠 破损;右前照灯总成 破损;右底大边 变形;右前翼子板 变形;右后翼子板...,NaN,NaN
2,CRP160331000670,右后翼子板 钣金修复;右后翼子板 钣金修复;右尾灯框架 钣金修复;左前翼子板 钣金修复;后围...,NaN,NaN
3,CRP160422000473,右前翼子板 钣金修复;右前翼子板内衬 钣金修复;左前翼子板 钣金修复;左前照灯框架 钣金修复...,NaN,NaN
4,CRP160319000073,发动机 异响;发动机运转 不平稳;发动机舱盖 新更换;左前照灯总成 新更换;右前照灯总成 新...,NaN,NaN
5,CRP160302000470,右后翼子板 重做漆;右前翼子板 重做漆;左后翼子板 重做漆;左前翼子板 重做漆;发动机 异响...,NaN,NaN
6,CRP160427000694,右后翼子板 钣金修复;右前翼子板 钣金修复;左后翼子板 钣金修复;左前翼子板 钣金修复;水箱...,NaN,NaN
7,CRP160224000537,右后翼子板 重做漆;发动机舱盖内边缘 钣金修复;右后车门封边 封边不整;左前车门封边 封边不...,NaN,NaN
8,CRP160319000277,"发动机 冒蓝烟,异响;车顶 变形;左后翼子板 变形,掉漆;右前翼子板 变形,掉漆;左前翼子板...",NaN,NaN
9,CRP160328000653,发动机舱盖内边缘 封边不整;自动档变速器 换挡有冲击;随车工具 缺失;后围板 划痕;发动机舱...,NaN,NaN


In [23]:
data_df_list_df.to_excel(curr_dir+'/检测报告数据_2020.xlsx')

